In [37]:
#!pip install -U openmim
#!mim install mmcv-full

In [38]:
#!pip install mmdet

In [39]:
#!pip install opencv-python

In [1]:
import os
import glob
import json
import mmcv
import numpy as np
from PIL import Image
from mmdet.apis import init_detector, inference_detector
from tqdm.notebook import tqdm

In [2]:
repo_dir = r'E:\coding\HimariO\HatefulMemesChallenge'
img_dir = os.path.join(repo_dir, r'data\hateful_memes\img_clean')
out_dir = os.path.join(repo_dir, r'data\hateful_memes\split_img_clean')
anno_out = os.path.join(repo_dir, r'data\hateful_memes\split_img_clean_boxes.json')

config_file = r'configs\faster_rcnn_r2_101_fpn_2x_img_clip\faster_rcnn_r2_101_fpn_2x_img_clip.py'
checkpoint_file=os.path.join(repo_dir, r'pretrain_model\faster_rcnn_r2_101_fpn_2x_img_clip\epoch_3.pth')

In [3]:
(img_dir,
out_dir,
anno_out,
config_file,
checkpoint_file)

('E:\\coding\\HimariO\\HatefulMemesChallenge\\data\\hateful_memes\\img_clean',
 'E:\\coding\\HimariO\\HatefulMemesChallenge\\data\\hateful_memes\\split_img_clean',
 'E:\\coding\\HimariO\\HatefulMemesChallenge\\data\\hateful_memes\\split_img_clean_boxes.json',
 'configs\\faster_rcnn_r2_101_fpn_2x_img_clip\\faster_rcnn_r2_101_fpn_2x_img_clip.py',
 'E:\\coding\\HimariO\\HatefulMemesChallenge\\pretrain_model\\faster_rcnn_r2_101_fpn_2x_img_clip\\epoch_3.pth')

In [4]:
imgs = glob.glob(os.path.join(img_dir, '*.png'))
os.makedirs(out_dir, exist_ok=True)

print(F"Find {len(imgs)} imgs!")
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

Find 10000 imgs!


C:\ProgramData\Anaconda3\envs\torchvision2\lib\site-packages\mmdet\models\builder.py:51: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load checkpoint from local path: E:\coding\HimariO\HatefulMemesChallenge\pretrain_model\faster_rcnn_r2_101_fpn_2x_img_clip\epoch_3.pth


In [29]:
model

FasterRCNN(
  (backbone): Res2Net(
    (stem): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Res2Layer(
      (0): Bottle2neck(
        (conv1): Conv2d(64, 104, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(104, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [5]:
def work(img, debug=False):
    # test a single image and show the results
    # img = 'test.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    if debug:
        pass
    else:
        det_img_clips = result[0]
        crops = []
        crops_anno = []
        for box in det_img_clips:
            if box[-1] > 0.5:
                crops.append(box)
                crops_anno.append(box.tolist())
            else:
                break
        img_name = os.path.basename(img)
        crop_anno[img_name] = crops_anno
        if len(crops) > 1:
            pil_img = Image.open(img)
            w, h = pil_img.size
            if w > h:
                crops = sorted(crops, key=lambda x: x[0])
                for j in range(len(crops) - 1):
                    crops[j][2] = (crops[j][2] + crops[j + 1][0]) / 2
            else:
                crops = sorted(crops, key=lambda x: x[1])
                for j in range(len(crops) - 1):
                    crops[j][3] = (crops[j][3] + crops[j + 1][1]) / 2

            for j, crop in enumerate(crops):
                print(crop)
                box = crop.astype(np.int32)
                img_name = os.path.basename(img)
                out_img_path = os.path.join(out_dir, img_name.replace('.png', f'.{j}.png'))
                pil_img.crop(
                    [max(box[0], 0),
                    max(box[1], 0),
                    min(box[2], w),
                    min(box[3], h),]
                ).save(out_img_path)
            # import pdb; pdb.set_trace()
        elif len(crops) == 1:
            pass
    if debug:
        model.show_result(img, result)

In [36]:
crop_anno = {}
for img in tqdm(imgs):
    work(img, False)

# visualize the results in a new window
# 
with open(anno_out, 'w') as f:
    json.dump(crop_anno, f)


  0%|          | 0/10000 [00:00<?, ?it/s]

[ 56.043682     7.6237054  643.7298     324.2381       0.99997044]
[  0.        315.29544   712.        800.          0.9999049]
[  0.           8.928322   336.29495    245.22021      0.99999905]
[ 49.085606  246.71281   304.39865   580.7399      0.9998223]
[  0.        577.50726   337.6911    794.49054     0.9999666]
[  3.3990822   0.        570.44086   449.14703     0.9999757]
[ 16.746307  443.41583   555.6737    800.          0.9999287]
[  3.2633712   0.        297.3872    200.93927     0.9996599]
[  3.4282134  197.14194    305.         399.           0.99975485]
[ 77.556625     0.         414.9356     491.50848      0.99968207]
[  0.         483.59357    506.         800.           0.99993694]
[  4.867481     6.2000394  291.94788    208.40604      0.99975437]
[  3.4943619  204.92976    293.79715    398.24658      0.99956924]
[4.7445536e+00 2.0352658e-02 2.9125964e+02 1.9956387e+02 9.9945229e-01]
[  3.5780792 196.48785   293.43823   400.          0.999762 ]
[  7.206814   9.852698 30

[1.1057541e+02 1.4192763e-01 4.2552536e+02 4.3397870e+02 9.9984360e-01]
[  0.9030971 428.4333    548.        800.          0.9999435]
[  0.          7.992144  326.        296.39508     0.9999559]
[  0.        290.66553   326.        800.          0.9999416]
[ 20.390614    0.        577.27386   413.47977     0.9935442]
[ 11.392143    26.95956    598.         420.0452       0.91192317]
[  0.          1.3462167 311.74454   449.34943     0.9999515]
[ 25.872904   445.9567     271.62122    787.73         0.99993443]
[3.8520148e-01 1.5452001e+00 5.9100000e+02 3.9522577e+02 9.9995542e-01]
[  0.         389.40863    589.6693     799.           0.99992764]
[  0.8349208    0.         634.         422.33215      0.99993503]
[164.21262   419.35266   464.8034    798.8822      0.9999876]
[ 16.532703     1.2744397  225.         233.11728      0.99346715]
[  8.502997  182.02722   225.        396.9977      0.9962214]
[ 23.798748    0.        542.88824   340.888       0.9998186]
[  0.        333.67654   

[  0.          5.072667  381.        248.03937     0.9999887]
[  7.6967106 242.38232   378.06848   800.          0.9996594]
[  5.683411     0.         652.         374.71234      0.99991906]
[  0.         370.85123    652.         800.           0.99994457]
[ 37.280117    1.420969  636.0088    307.12576     0.9999893]
[  2.3542223 304.94644   740.00006   800.00006     0.9999348]
[ 59.96192     6.7825656 611.63635   350.91962     0.9999809]
[  0.         349.4106     671.         799.           0.99993813]
[  8.644418    0.        292.09106   201.47034     0.9990829]
[  0.        192.61478   299.        400.          0.9996351]
[  0.           3.9579957  256.4951     238.27611      0.85487604]
[ 59.592358     0.         615.2743     246.           0.98859197]
[4.0554858e+02 8.9199114e+00 8.0777032e+02 2.2487158e+02 5.9023279e-01]
[  0.         10.475028  754.        419.52286     0.9998411]
[ 81.620476  417.27417   665.7257    800.          0.9999559]
[  0.          0.        372.      

[1.5635477e-01 8.2842426e+00 4.2400000e+02 2.1144618e+02 9.7861266e-01]
[ 10.06737     74.324615   420.8673     800.           0.99923074]
[  3.2609694   0.        413.6776    533.9976      0.9999391]
[  0.         522.6503     422.         800.           0.99984586]
[  0.         156.17188    242.         479.12964      0.99999154]
[1.0244062e-02 4.8240378e+02 2.4200000e+02 6.4636664e+02 9.9999607e-01]
[  3.7750142    0.30670452 289.4776     196.86314      0.9998079 ]
[  0.        190.75215   299.        398.23032     0.9998864]
[  0.          0.        615.        401.05817     0.9997993]
[111.124756 403.71704  506.64682  797.2225     0.999754]
[5.9172708e-01 0.0000000e+00 7.3000000e+02 5.0964221e+02 9.9995995e-01]
[129.55122   501.31448   596.99146   799.99994     0.9999988]
[ 66.54084      4.1137967  643.64124    326.01703      0.99999774]
[  0.         319.8095     712.         800.           0.99992716]
[  0.         1.330777 283.       400.9137     0.999941]
[  0.         395.39

...

[  2.518903     7.917968   132.1603     191.43051      0.99823964]
[  0.         197.18607    131.99612    390.93198      0.99103165]
[  0.          2.5197384 602.80493   401.62146     0.999933 ]
[  0.87211627 397.37622    602.80225    800.           0.99991214]
[  0.           2.166931   599.         397.79486      0.99994206]
[  0.         392.9572     597.59845    798.82874      0.99990845]
[153.54646     0.        468.02176   373.3614      0.9999969]
[  0.        375.10504   621.        799.99994     0.9999249]
[  1.526577    2.6157088 617.4451    386.59735     0.9999232]
[  0.        382.91174   625.3963    800.          0.9999448]
[  6.001312     1.6609163  401.25717    235.3285       0.99999547]
[ 28.546177   235.4608     391.02832    472.23575      0.99997747]
[  0.         469.5236     418.         800.           0.99993086]
[  0.          1.4121656 703.        248.47162     0.999856 ]
[0.00000000e+00 1.09509865e+02 7.03000000e+02 4.85173279e+02
 6.68911815e-01]
[ 13.5687065  

[  0.          0.        683.        458.75043     0.9999516]
[ 61.01173   452.0261    609.879     798.3699      0.9997074]
[  0.           0.         511.85864    509.80316      0.99996114]
[198.7173   501.8727   627.0174   775.         0.999992]
[  1.0289446    0.90832037 313.         400.8158       0.99994767]
[2.5792235e-01 3.9584497e+02 3.1300000e+02 8.0000000e+02 9.9994266e-01]
[  0.          2.220068  417.        286.66052     0.9998833]
[  2.9667299  280.21506    417.         800.           0.99996316]
[195.0422      1.7957249 587.07227   279.02222     0.9999939]
[  0.        274.01996   786.99994   800.          0.9999505]
[ 14.499628    0.        387.5742    400.5226      0.5405639]
[  0.           1.0452442  403.         298.90094      0.99993086]
[  0.        291.14127   403.        551.41174     0.9998204]
[ 30.082209  552.00385   370.20996   800.          0.9999908]
[ 65.00254      1.385961   643.8149     274.58533      0.99990857]
[  0.        270.40915   739.59973   800

[104.205864    0.        413.21265   424.52435     0.9995926]
[  0.        422.84897   530.64166   799.          0.9999502]
[ 52.95545      0.         447.5085     486.1233       0.99918467]
[ 12.706981   442.12692    506.         778.16925      0.95166165]
[ 10.699549     0.         468.37967    154.75984      0.99958986]
[  9.903351    59.936485   451.56433    319.62866      0.63127583]
[ 38.47427   254.27237   429.4204    483.41678     0.9998035]
[  1.4636902 483.50797   474.        800.          0.9999429]
[  2.0209901   0.        467.1476    474.48575     0.999972 ]
[  0.         473.00848    474.         800.           0.99994874]
[ 30.455072     0.         449.43628    515.4182       0.99994636]
[ 22.443586  512.8356    449.70288   783.7908      0.9996414]
[  0.           1.8131058  367.         511.49536      0.99997306]
[ 81.34244    506.5388     287.76605    796.149        0.99998856]
[  2.7975955   0.        191.        250.29948     0.9844707]
[  0.         173.17023    191

[  0.           4.599285   403.         261.34827      0.99988854]
[  0.        256.1699    401.64633   800.          0.9999157]
[  3.176857     0.40661883 299.85162    208.6076       0.9999186 ]
[  3.6990166  205.46779    298.96326    400.           0.99980325]
[  0.          2.1990743 145.        149.7899      0.9992434]
[  4.859803  147.96631   142.03268   273.09393     0.9224223]
[  6.9991264 268.14343   145.        396.10577     0.9991968]
[ 13.143827    0.        403.459     436.03317     0.9999155]
[  4.6395645 435.03986   414.46844   798.7558      0.9999099]
[7.4453396e-01 1.4001077e-01 5.7707648e+02 3.9695099e+02 9.9994147e-01]
[  0.        390.05832   579.        793.05676     0.9998914]
[  0.           0.         614.         396.74588      0.99994576]
[  0.        389.9602    614.        800.          0.9999273]
[  1.1857224    0.8755885  600.         401.51526      0.99993515]
[  0.        397.15863   598.47296   800.          0.9999231]
[  0.           0.86263466 703.    

[  6.246895     0.         291.85046    198.77393      0.99962974]
[  9.527229  196.32109   296.2661    400.          0.9997867]
[  0.           0.         299.3253     202.70107      0.99992514]
[  0.        199.83315   300.        400.          0.9998529]
[  1.6979027   0.        630.        420.45425     0.999943 ]
[1.6717070e-01 4.1191171e+02 6.2162195e+02 8.0000000e+02 9.9989903e-01]
[  1.1307951   1.6768113 523.        352.11566     0.9999567]
[ 92.87661   342.79285   414.6727    800.          0.9996517]
[ 64.17801      4.9536963  408.58624    207.77817      0.99999547]
[  7.652745  199.22604   480.03406   498.44568     0.9976763]
[ 17.88416   369.24057   471.03687   800.          0.9642139]
[  2.8133464  35.208054  381.07336   166.84877     0.9998516]
[  1.2759043  45.330963  375.56723   486.78546     0.7673617]
[  6.908588   284.1994     381.61926    526.731        0.99998283]
[ 19.387388  525.86847   365.4933    784.2837      0.9999968]
[  1.9801036  11.813714  133.        191

In [6]:
image_name = '01236.png'
work(os.path.join(img_dir, image_name), True)

C:\ProgramData\Anaconda3\envs\torchvision2\lib\site-packages\mmdet\datasets\utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
